# Bibliotecas

In [2]:
!pip install reppy

In [3]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
#import urllib.robotparser as robot
from requests.exceptions import HTTPError
from reppy.robots import Robots
import urllib

In [16]:
import sys
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl, QEventLoop
from PyQt5.QtWebEngineWidgets import QWebEnginePage

In [93]:
from selenium import webdriver
from reppy.robots import Robots
from urllib.parse import urlparse

# Crawler

In [94]:
Dominio = ['https://www.capefeargames.com/','https://www.mtgotraders.com/',
           'https://www.wizardscupboard.com/','https://abugames.com/',
           'https://www.cardkingdom.com/','http://www.starcitygames.com/',
           'https://www.tcgplayer.com/','https://scryfall.com/']

paginasVisitadas = []

paginasRelevante = {'Relevate':[],'Nao-Relevate':[]}

In [95]:
# Função para marcar a page como visitada.
def Visitou (url):
    if url not in paginasVisitadas:
        #paginasVisitdas.append(url)
        return False
    else:
        return True

In [96]:
# Função para verificar se a requisição sofreu algum problema.
def verificarRequest(url):
    try:
        response = rq.head(url)
        # Se o response retornou cod = 200 (sucesso), não teremos nenhuma exceção
        response.raise_for_status()

    except HTTPError as http_err:
        #print(f'HTTP error occurred: {http_err}')
        return False
    except Exception as err:
        #print(f'Other error occurred: {err}')
        return False
    return True

In [97]:
def CrawlerDinâmico(url):
    driver = webdriver.Chrome("C:\\Users\\jcss3\\Downloads\\chromedriver_win32 _76\\chromedriver.exe")
    driver.get(url)
    res = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    return res

In [98]:
#Função para Verificar se podemos baixar a page ou não
def verificarRobotTxt(url):
    
    # This utility uses `requests` to fetch the content
    robots = Robots.fetch(url + '/robots.txt')
    return robots.allowed(url, '*')

In [99]:
verificarRobotTxt(Dominio[0])

True

## * Baseline

In [122]:
def CardsCrawler(url):
    print('Dominio: ' + url.geturl())
    count = 0
    paginas = [url]
    #paginasVisitadas = []
    robots = Robots.fetch(url.geturl() + '/robots.txt')
    while count < 1000 and len(paginas) > 0:
        count = count + 1
        print(count)
        primeiraPagina = paginas.pop(0)
        print(primeiraPagina.geturl())
       
        # Verifica se a pagina foi visitada.Se foi, pego o proximo link da lista.(continua até achar um que não foi visitado!)
        while Visitou(primeiraPagina) == True or primeiraPagina.hostname != url.hostname or primeiraPagina.hostname == None:
            if len(paginas) == 0:
                print('lascou@!')
                return
            primeiraPagina = paginas.pop(0)
        
        if robots.allowed(primeiraPagina.geturl(), '*') == True:
            # verificar se a requisição funciana.
            if verificarRequest(primeiraPagina.geturl()) == True:

                # Verficar se o site é dinamico
                if (Dominio[3] in url) or (Dominio[6] in url):
                    texto = CrawlerDinâmico(primeiraPagina.geturl())
                else:
                    #request - verificando se a page é html pelo content-type.
                    codigo_fonte = rq.get(primeiraPagina.geturl(),headers={'content-type': 'text/html'})
                    # texto,links dentro do codigo_fonte.
                    texto = codigo_fonte.text
                soup = BeautifulSoup(texto,'html.parser')

                # Laço para encontrar todos os links da pagina(url). # attrs={'href': re.compile("^http://")}
                for link in soup.findAll('a'):
                    href = link.get('href')
                    href = urllib.parse.urljoin(url.geturl(),href)
                    paginas.append(urlparse(href))

                #marco como visitada a url    
                paginasVisitadas.append(primeiraPagina)

            else:
                print('Requisição falhou!')
        else:
            print('Respeitando o Robot.txt não iremos ')

In [ ]:
for url in Dominio:
    CardsCrawler(urlparse(url))

Dominio: https://www.capefeargames.com/
1
https://www.capefeargames.com/
lascou@!
Dominio: https://www.mtgotraders.com/
1
https://www.mtgotraders.com/
lascou@!
Dominio: https://www.wizardscupboard.com/
1
https://www.wizardscupboard.com/
2
https://www.wizardscupboard.com
3
https://www.wizardscupboard.com/index.php
4
https://www.wizardscupboard.com/account.php
Respeitando o Robot.txt não iremos 
5
https://www.wizardscupboard.com/shopping_cart.php
Respeitando o Robot.txt não iremos 
6
https://www.wizardscupboard.com/checkout_shipping.php
Respeitando o Robot.txt não iremos 
7
https://www.wizardscupboard.com/advanced_search.php
8
https://www.wizardscupboard.com/singles-c-100.html
9
https://www.wizardscupboard.com/foils-c-1100.html
10
https://www.wizardscupboard.com/sets-c-500.html
11
https://www.wizardscupboard.com/booster-boxes-c-210.html
12
https://www.wizardscupboard.com/booster-packs-c-220.html
13
https://www.wizardscupboard.com/other-sealed-product-c-230.html


In [104]:
urlparse('mailto:info@capefeargames.com').hostname == None

True

## * Heurística